In [1]:
import pandas as pd
import numpy as np
import pickle as pkl

import config

In [3]:
test_cols = ['주야', '요일','사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수', '발생지시도',
             '발생지시군구', '사고유형_대분류', '사고유형_중분류', '법규위반', '도로형태_대분류', '도로형태',
             '당사자종별_1당_대분류', '당사자종별_2당_대분류']

In [4]:
config.col_dict

{'A': '주야',
 'B': '요일',
 'C': '사망자수',
 'D': '사상자수',
 'E': '중상자수',
 'F': '경상자수',
 'G': '부상신고자수',
 'H': '발생지시도',
 'I': '발생지시군구',
 'J': '사고유형_대분류',
 'K': '사고유형_중분류',
 'L': '법규위반',
 'M': '도로형태_대분류',
 'N': '도로형태',
 'O': '당사자종별_1당_대분류',
 'P': '당사자종별_2당_대분류'}

In [5]:
df_test = pd.read_csv('./data/test_kor.csv', encoding='cp949')

In [6]:
df_result = pd.read_csv('./data/result_kor.csv', encoding='cp949')

In [5]:
df_train = pd.read_csv('./data/dataset_kor/교통사망사고정보/Kor_Train_교통사망사고정보(12.1~17.6).csv', engine='python')

In [6]:
df = df_train[test_cols]
df.tail(3)

,주야,요일,사망자수,사상자수,중상자수,경상자수,부상신고자수,발생지시도,발생지시군구,사고유형_대분류,사고유형_중분류,법규위반,도로형태_대분류,도로형태,당사자종별_1당_대분류,당사자종별_2당_대분류
25034,야간,일,1,1,0,0,0,서울,서초구,차대사람,횡단중,안전운전 의무 불이행,교차로,교차로부근,승용차,보행자
25035,야간,일,1,6,5,0,0,전북,정읍시,차대차,정면충돌,중앙선 침범,단일로,기타단일로,승용차,승용차
25036,야간,일,1,1,0,0,0,서울,은평구,차대사람,차도통행중,안전운전 의무 불이행,단일로,기타단일로,승용차,보행자


In [36]:
kv_map = {}

kv_map['주야'] = {'N':0, '주간':1, '야간':2}

kv_map['요일'] = {'N':0, '일':1, '월':2, '화':3, '수':4, '목':5, '금':6, '토':7, '일':8}

kv_map['발생지시도'] = {'N':0, '서울':1, '부산':2, '대전':3, '대구':4, '광주':5, '인천':6, '울산':7, '세종':8,
                      '경기':9, '강원':10, '충남':11, '충북':12, '전남':13, '전북':14, '경남':15, '경북':16, '제주':17}

list_bj = ['N', '강남구','강동구','강북구','강서구','관악구','광진구','구로구','금천구','노원구',
            '도봉구','동대문구','동작구','마포구','서대문구','서초구','성동구','성북구','송파구',
            '양천구','영등포구','용산구','은평구','종로구','중구','중랑구',
            '강서구','금정구','기장군','남구','동구','동래구','북구','사상구','사하구','서구','수영구','연제구',
            '영도구','중구','진구','해운대구','남구','달서구','달성군','동구','북구','서구','수성구','중구',
            '강화군','계양구','남구','남동구','동구','부평구','서구','연수구','옹진군','중구',
            '광산구','남구','동구','북구','서구','대덕구','대전','동구','서구','유성구','중구',
            '남구','동구','북구','울주군','중구','세종특별자치시','가평군','강화군','고양시','과천시','광명시',
            '광주시','구리시','군포시','김포시','남양주군','남양주시','동두천시','부천시','성남시','송탄시','수원시',
            '시흥시','안산시','안성시','안양시','양주시','양평군','여주시','연천군','오산시','옹진군',
            '용인시','의왕시','의정부시','이천시','파주시','평택군','평택시','포천시','하남시','화성시','강릉시',
            '고성군','동해시','명주군','삼척군','삼척시','속초시','양구군','양양군','영월군','원성군','원주시','인제군','정선군','철원군','춘성군',
            '춘천시','태백시','평창군','홍천군','화천군','횡성군','괴산군','단양군','보은군','영동군','옥천군',
            '음성군','제원군','제천시','중원군','증평군','진천군','청원군','청주시','충주시','계룡시',
            '공주군','공주시','금산군','논산시','당진시','대전시','대천시','보령시','부여군','서산군','서산시','서천군',
            '아산시','연기군','예산군','천안시','천원군','청양군','태안군','홍성군','고창군','군산시','김제군',
            '김제시','남원군','남원시','무주군','부안군','순창군','옥구군','완주군','익산군','익산시','임실군',
            '장수군','전주시','정읍군','정읍시','진안군','강진군','고흥군','곡성군','광양시','광양시','구례군',
            '김제시','나주군','나주시','담양군','목포시','무안군','보성군','순천시','승주군','신안군',
            '여수시','여천시','영광군','영암군','완도군','장성군','장흥군','진도군','함평군','해남군','화순군',
            '경산군','경산시','경주군','경주시','고령군','구미시','군위군','금릉군','김천시','달성군',
            '문경군','문경시','봉화군','상주군','상주시','선산군','성주군','안동군','안동시','영덕군',
            '영양군','영일군','영주시','영천군','영천시','영풍군','예천군','울릉군','울진군','의성군',
            '청도군','청송군','칠곡군','포항시','거제군','거제시','거창군','고성군','김해군','김해시','남해군',
            '마산시','밀양군','밀양시','사천군','사천시','산청군','양산시','울산군','울산시','의령군','진양군',
            '진주시','진해시','창녕군','창원군','창원시','창원시(통합)','통영군','통영시','하동군','함안군',
            '함양군','합천군','남제주군','북제주군','서귀포시','제주시']


kv_map['발생지시군구'] = {v:i for i, v in enumerate(pd.Series(list_bj).unique())} 

kv_map['사고유형_대분류'] = {'N':0, '차대사람':1, '차대차':2, '차량단독':3, '건널목':4}

kv_map['사고유형_중분류'] = {'N':0, '측면충돌':1, '차도통행중':2, '전도전복':3, '정면충돌':4,
                           '추돌':5, '횡단중':6, '보도통행중':7, '공작물충돌':8, '기타':9,
                           '전도':10, '도로이탈':11, '길가장자리구역통행중':12, '전복':13,
                           '후진중충돌':14, '주/정차차량 충돌':15, '측면직각충돌':16, 
                           '차단기돌파':17, '직전진행':18, '경보기무시':19}

kv_map['법규위반'] = {'N':0, '안전운전 의무 불이행':1, '중앙선 침범':2, '안전거리 미확보':3, '과속':4,
                     '신호위반':5, '직진 및 우회전차의 통행방해':6, '교차로 통행방법 위반':7, '보행자 보호의무 위반':8,
                     '부당한 회전':9, '기타(운전자법규위반)':10, '차로위반(진로변경 위반)':11, '앞지르기 금지위반':12,
                     '앞지르기 방법위반':13, '정비불량 제차의 운전금지위반':14, '서행 및 일시정지위반':15, '과로':16,
                     '철길건널목 통과방법위반':17, '진로양보 의무 불이행':18, '보행자과실':19, '통행우선 순위위반':20}

kv_map['도로형태_대분류'] = {'N':0, '단일로':1, '교차로':2, '기타':3, '주차장':4, '불명':5,
                           '기타/불명':6, '고가도로위':7, '지하도로내':8, '건널목':9}

kv_map['도로형태'] = {'N':0, '기타단일로':1, '교차로부근':2, '교량위':3, '교차로내':4, '고가도로위':5,
                     '교차로횡단보도내':6, '기타':7, '지하차도(도로)내':8, '주차장':9, '터널안':10,
                     '불명':11, '횡단보도상':12, '횡단보도부근':13, '기타/불명':14, '지하도로내':15, '건널목':16}

kv_map['당사자종별_1당_대분류'] = {'N':0, '승용차':1, '자전거':2, '화물차':3, '승합차':4, '건설기계':5,
                     '이륜차':6, '원동기장치자전거':7, '특수차':8, '사륜오토바이(ATV)':9, '농기계':10,
                     '개인형이동수단(PM)':11, '불명':12}

kv_map['당사자종별_2당_대분류'] = {'N':0, '승합차':1, '보행자':2, '없음':3, '화물차':4, '특수차':5,
                     '승용차':6, '원동기장치자전거':7, '이륜차':8, '농기계':9, '자전거':10,
                     '사륜오토바이(ATV)':11, '건설기계':12, '불명':13, '열차':14}

In [32]:
for key in kv_map.keys():
    print(key)
    df[key] = df[key].apply(lambda x: kv_map[key][x])

주야
요일
발생지시도


c:\users\timmy\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
df_out =df[df.columns[:8]]

In [40]:
df_out.to_csv('./encoded_df.csv', index=False, encoding = 'cp949')

In [42]:
with open('./kv_map.pkl', 'wb') as f:
    pkl.dump(kv_map, f)